<a href="https://colab.research.google.com/github/leeh-nix/air-quality-index-prediction/blob/main/AQI_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import pathlib
from sklearn.model_selection import train_test_split


In [2]:
# dataset_path = pathlib.Path(
#     "/content/drive/MyDrive/"
# )
df = pd.read_csv("weather_dataset.csv")
dataset = df

In [3]:
columns_to_drop = [
    "last_updated",
    "last_updated_epoch",
    "wind_direction",
    "sunrise",
    "sunset",
    "moonrise",
    "moonset",
    "moon_phase",
    "moon_illumination",
    "feels_like_celsius",
    "feels_like_fahrenheit",
    "precip_in",
    "temperature_fahrenheit",
    "pressure_in",
    "timezone",
]
dataset = dataset.drop(columns=columns_to_drop)


us_dataset = dataset.drop(
    columns=["air_quality_gb-defra-index", "visibility_km", "gust_kph", "wind_kph"]
)


gb_dataset = dataset.drop(
    columns=["air_quality_us-epa-index", "visibility_miles", "gust_mph", "wind_mph"]
)

In [4]:
print(df.columns)
print(dataset.columns)
print(us_dataset.columns)
print(gb_dataset.columns)


Index(['country', 'location_name', 'region', 'latitude', 'longitude',
       'timezone', 'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')
Index(['country', 'location_name', 'region', 'latitude', 'longitude',
       'temperature_celsius', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'pressure

In [5]:
# gb_dataset = gb_dataset.drop(columns=[""])
gb_dataset.columns

Index(['country', 'location_name', 'region', 'latitude', 'longitude',
       'temperature_celsius', 'condition_text', 'wind_kph', 'wind_degree',
       'pressure_mb', 'precip_mm', 'humidity', 'cloud', 'visibility_km',
       'uv_index', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_gb-defra-index'],
      dtype='object')

In [6]:
X = gb_dataset.iloc[:, :-1]
y = gb_dataset.iloc[:, -1]
print(X.columns)
print(y)

Index(['country', 'location_name', 'region', 'latitude', 'longitude',
       'temperature_celsius', 'condition_text', 'wind_kph', 'wind_degree',
       'pressure_mb', 'precip_mm', 'humidity', 'cloud', 'visibility_km',
       'uv_index', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10'],
      dtype='object')
0         2
1         1
2         2
3         1
4         1
         ..
74438    10
74439     2
74440    10
74441     9
74442    10
Name: air_quality_gb-defra-index, Length: 74443, dtype: int64


In [7]:
from sklearn.preprocessing import OneHotEncoder

# Assuming your DataFrame is named 'df'
# Select the columns to one-hot encode
columns_to_encode = ["country", "location_name", "region", "condition_text"]

# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse_output=False, drop="first")

# Fit and transform the selected columns
encoded_columns = encoder.fit_transform(X[columns_to_encode])

# Create a DataFrame from the encoded columns
X_encoded = pd.DataFrame(
    encoded_columns, columns=encoder.get_feature_names_out(columns_to_encode)
)

# Concatenate the original DataFrame with the new DataFrame
X = pd.concat([X, X_encoded], axis=1)

# Drop the original categorical columns
X = X.drop(columns=columns_to_encode)

# Display the resulting DataFrame
print(X.head())

   latitude  longitude  temperature_celsius  wind_kph  wind_degree  \
0     24.57      77.72                 27.5      20.5          281   
1     23.33      77.80                 27.5      15.5          287   
2     22.07      78.93                 26.3      18.4          317   
3     21.86      77.93                 25.6      16.9          297   
4     22.75      77.72                 27.2      16.2          274   

   pressure_mb  precip_mm  humidity  cloud  visibility_km  ...  \
0       1008.0        0.0        67     26           10.0  ...   
1       1008.0        0.0        70     19           10.0  ...   
2       1009.0        0.0        70     51           10.0  ...   
3       1009.0        0.0        76     65           10.0  ...   
4       1009.0        0.0        74     82           10.0  ...   

   condition_text_Patchy light rain  \
0                               0.0   
1                               0.0   
2                               0.0   
3                         

In [8]:
X.shape

(74443, 631)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(59554, 631) (14889, 631) (59554,) (14889,)
